In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict
import sys

In [2]:
def get_x():
    x_pt = torch.rand((1,3,8,224,224))
    x_np = x_pt.numpy()
    x_tf = tf.convert_to_tensor(x_np)

    return x_tf, x_pt

## Convert Weights

Initiate model and Load PyTorch Weights

In [3]:
from torch_intermediate_layer_getter import IntermediateLayerGetter as MidGetter


cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

pt_model = SwinTransformer3D_pt(**cfg, isTest= True)
tf_model = SwinTransformer3D(**cfg, isTest= True)
x_tf, x_pt = get_x()


basic_pt, y = tf_model(x_tf)

basic_tf, z= pt_model(x_pt)

checkpoint = torch.load(f'{name}.pth')



new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    if 'backbone' in k:
        name = k[9:]
        new_state_dict[name] = v 

pt_model.load_state_dict(new_state_dict) 


c:\Python\Python396\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


<All keys matched successfully>

Convert Functions

In [4]:
def conv_transpose(w):
    return w.transpose(2,3,4,1, 0)
    

def modify_tf_block( tf_component, pt_weight,  pt_bias = None, is_attn=False):
    in_shape = pt_weight.shape

    if isinstance(tf_component, tf.keras.layers.Conv3D) :
      pt_weight = conv_transpose(pt_weight)

    if isinstance(tf_component, tf.keras.layers.Dense) and not is_attn:
      pt_weight =pt_weight.transpose()

    if isinstance(tf_component, (tf.keras.layers.Dense, tf.keras.layers.Conv3D)):
        tf_component.kernel.assign(tf.Variable(pt_weight))

        if pt_bias is not None:
            tf_component.bias.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, tf.keras.layers.LayerNormalization):

        tf_component.gamma.assign(tf.Variable(pt_weight))

        tf_component.beta.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, (tf.Variable)):
        tf_component.assign(tf.Variable(pt_weight))

    else:
        return tf.convert_to_tensor(pt_weight)
        
        

    return tf_component


def modify_swin_blocks(np_state_dict, pt_weights_prefix, tf_block):

  for layer in tf_block:
    if isinstance(layer, PatchMerging):
      patch_merging_idx = f"{pt_weights_prefix}.downsample"

      layer.reduction = modify_tf_block( layer.reduction,
                          np_state_dict[f"{patch_merging_idx}.reduction.weight"])
      layer.norm = modify_tf_block( layer.norm,
                        np_state_dict[f"{patch_merging_idx}.norm.weight"],
                        np_state_dict[f"{patch_merging_idx}.norm.bias"]
                        )
      
  # Swin Layers
  common_prefix = f"{pt_weights_prefix}.blocks"
  block_idx = 0

  for outer_layer in tf_block:

      layernorm_idx = 1
      mlp_layer_idx = 1

      if isinstance(outer_layer, SwinTransformerBlock3D):
          for inner_layer in outer_layer.layers:
        
              # Layer norm.
              if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                  layer_norm_prefix = (
                      f"{common_prefix}.{block_idx}.norm{layernorm_idx}"
                  )
                  inner_layer.gamma.assign(
                      tf.Variable(
                          np_state_dict[f"{layer_norm_prefix}.weight"]
                      )
                  )



                  inner_layer.beta.assign(
                      tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                  )

                  layernorm_idx += 1

              # Window attention.
              elif isinstance(inner_layer, WindowAttention3D):
                  attn_prefix = f"{common_prefix}.{block_idx}.attn"

                  # Relative position.
                  inner_layer.relative_position_bias_table = (
                      modify_tf_block(
                          inner_layer.relative_position_bias_table,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_bias_table"
                          ] 
                      )
                  )
                  inner_layer.relative_position_index = (
                      modify_tf_block(
                          inner_layer.relative_position_index,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_index"
                          ]
                      )
                  )

                  # QKV.
                  inner_layer.qkv = modify_tf_block(
                      inner_layer.qkv,
                      np_state_dict[f"{attn_prefix}.qkv.weight"],
                      np_state_dict[f"{attn_prefix}.qkv.bias"]
                  )

                  # Projection.
                  inner_layer.proj = modify_tf_block(
                      inner_layer.proj,
                      np_state_dict[f"{attn_prefix}.proj.weight"],
                      np_state_dict[f"{attn_prefix}.proj.bias"]
                  )

              # MLP.
              elif isinstance(inner_layer, tf.keras.Model):
                  mlp_prefix = f"{common_prefix}.{block_idx}.mlp"
                  for mlp_layer in inner_layer.layers:
                      if isinstance(mlp_layer, tf.keras.layers.Dense):
                          mlp_layer = modify_tf_block(
                              mlp_layer,
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                              ],
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                              ]
                          )
                          mlp_layer_idx += 1

          block_idx += 1
  return tf_block


### convert

In [5]:
np_state_dict = pt_model.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}

tf_model.projection.layers[0] = modify_tf_block(tf_model.projection.layers[0]
        ,
        np_state_dict["patch_embed.proj.weight"],
        np_state_dict["patch_embed.proj.bias"])

tf_model.projection.layers[1] = modify_tf_block(
    tf_model.projection.layers[1],
    np_state_dict["patch_embed.norm.weight"],
    np_state_dict["patch_embed.norm.bias"])


layer_normalization_idx = -1

tf_model.layers[layer_normalization_idx] = modify_tf_block(
    tf_model.layers[layer_normalization_idx] ,
    np_state_dict["norm.weight"],
    np_state_dict["norm.bias"]
    )

# swin layers
for i in range(2, len(tf_model.layers) - 1):
    _ = modify_swin_blocks(np_state_dict,
                        f"layers.{i-2}",
                        tf_model.layers[i].layers)

In [6]:
basic_pt, y = tf_model(x_tf)
basic_tf, z= pt_model(x_pt)

y.shape, z.shape

(TensorShape([1, 768, 4, 7, 7]), torch.Size([1, 768, 4, 7, 7]))

In [7]:
for layer in basic_pt:
    print("--------------",layer, "-------------\n TF: ", basic_tf[layer].detach().numpy(), "\n PT: ", basic_pt[layer].numpy(), "\n" )

-------------- PatchEmbed -------------
 TF:  [[[[[0.4694879  0.7320849  0.668663   0.6526338  0.8030871  0.9637686
     0.8512107  0.37930787 0.40388405 0.52170944]]]]] 
 PT:  [[[[[0.46948713 0.73208445 0.66866344 0.6526342  0.8030863  0.9637688
     0.8512116  0.37930882 0.40388483 0.5217101 ]]]]] 

-------------- basic layer1 -------------
 TF:  [[[[[ 0.05090611  0.6119843   0.19952458 -0.26986203  0.43620658
      0.1511359  -0.08458254  0.49404496  0.46321994 -0.40098152]]]]] 
 PT:  [[[[[ 0.03283307  0.5923909   0.1710603  -0.2625764   0.43319678
      0.14306492 -0.06784578  0.4730916   0.45867777 -0.4456384 ]]]]] 

-------------- basic layer2 -------------
 TF:  [[[[[-0.56932425 -0.5508435  -0.63364273 -0.47360727 -0.49055067
     -0.1942482  -0.56187135 -0.3953247  -0.44955727 -0.4444399 ]]]]] 
 PT:  [[[[[-0.5480101  -0.5739908  -0.65543723 -0.46185446 -0.46572882
     -0.23131174 -0.5980027  -0.35696417 -0.46548173 -0.46797377]]]]] 

-------------- basic layer3 -------------
 

### PT basic layer outputs comparison

In [8]:
attempts = 10
x_pt = torch.rand((1,3,8,224,224))

outputs = []

for i in range(attempts):
    layer_out , result = pt_model(x_pt)
    outputs.append(layer_out)

i = 0
for layer in outputs[0]:
    print("--------------", layer, "---------------")
    for idx, layer_out in enumerate(outputs) :
        print(f"attempt {idx} : ",layer_out[layer].detach().numpy(), "\n")
    print()



-------------- PatchEmbed ---------------
attempt 0 :  [[[[[0.34786695 1.3482637  0.3097511  0.5879744  0.7180053  0.36379492
     0.46565378 0.6328912  0.41669297 1.0991832 ]]]]] 

attempt 1 :  [[[[[0.34786695 1.3482637  0.3097511  0.5879744  0.7180053  0.36379492
     0.46565378 0.6328912  0.41669297 1.0991832 ]]]]] 

attempt 2 :  [[[[[0.34786695 1.3482637  0.3097511  0.5879744  0.7180053  0.36379492
     0.46565378 0.6328912  0.41669297 1.0991832 ]]]]] 

attempt 3 :  [[[[[0.34786695 1.3482637  0.3097511  0.5879744  0.7180053  0.36379492
     0.46565378 0.6328912  0.41669297 1.0991832 ]]]]] 

attempt 4 :  [[[[[0.34786695 1.3482637  0.3097511  0.5879744  0.7180053  0.36379492
     0.46565378 0.6328912  0.41669297 1.0991832 ]]]]] 

attempt 5 :  [[[[[0.34786695 1.3482637  0.3097511  0.5879744  0.7180053  0.36379492
     0.46565378 0.6328912  0.41669297 1.0991832 ]]]]] 

attempt 6 :  [[[[[0.34786695 1.3482637  0.3097511  0.5879744  0.7180053  0.36379492
     0.46565378 0.6328912  0.41669